In [4]:
%pylab inline
%load_ext autoreload
%autoreload 2
from inpainting.dataset import Data, ResizeTransform, NoiseSampler
from gan.gan import Generator5Net, Discriminator5
from gan.trainer import GanTrainer
import torch
print(torch.__version__)
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt
import pandas as pd
from inpainting.visualize import plot_batch
from inpainting.visualize import GanPlotLossCallback as PlotLossCallback
from inpainting import celeba_config as conf
from inpainting.visualizer import Visualizer
from performance.estimator import FIDEstimator

%matplotlib notebook

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.4.0


ImportError: cannot import name 'random_split'

In [ ]:
torch.cuda.set_device(conf.CUDA_DEVICE)

In [ ]:

transform = ResizeTransform()
data = Data(conf.DATA_PATH, transform)
train_size = int(0.8 * len(data))
valid_size = len(data) - train_size
train_data, valid_data = torch.utils.data.random_split(data, [train_size, valid_size])
train_loader = DataLoader(train_data, batch_size=conf.BATCH_SIZE, num_workers=conf.NUM_WORKERS, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=conf.BATCH_SIZE, num_workers=conf.NUM_WORKERS, shuffle=True)
print('Dataset size: ', len(data))
noise_sampler = NoiseSampler(conf.Z_SIZE)


In [ ]:
estimator = FIDEstimator()
X1, = next(iter(train_loader))
X2, = next(iter(valid_loader))
distance = estimator.distance(X1, X2)
print(distance)
del X1, X2, estimator

In [ ]:
generator = Generator5Net(conf.Z_SIZE).cuda()
discriminator = Discriminator5().cuda()

In [ ]:
visualizer = Visualizer(conf, noise_sampler)
trainer = GanTrainer(generator, discriminator, conf, noise_sampler, visualizer=visualizer)

In [ ]:
if conf.CONTINUE_TRAINING:
    trainer.load_last_checkpoint()
    
trainer.train(train_loader, valid_loader, n_epochs=10)

In [ ]:

#print(Z)

In [ ]:
LOAD_MODEL = True
LOAD_EPOCH_N = 2
if LOAD_MODEL:
    generator.load_state_dict(torch.load(conf.MODEL_PATH + 'generator_%d.pth' % (LOAD_EPOCH_N,)))
    discriminator.load_state_dict(torch.load(conf.MODEL_PATH + 'discriminator_%d.pth' % (LOAD_EPOCH_N,)))

In [ ]:
for batch,in train_loader:
    plot_batch((batch.data.cpu().numpy() + 1)/2, limit=6)
    break

In [ ]:
Z, = noise_sampler.sample()
G_sample = generator(Z)
sample = G_sample.data.cpu().numpy()
print(discriminator.layer4.weight.cpu().detach().numpy())
plot_batch((G_sample.data.cpu().numpy() + 1) / 2)

In [ ]:
w = generator.layer3[0].weight.data.cpu().numpy()
print(w)

In [ ]:
for X, in valid_loader:
    Z, = noise_sampler.sample_batch(valid_loader.batch_size)
    G_sample = generator(Z)
    D_real, D_logit_real = discriminator(X)
    D_fake, D_logit_fake = discriminator(G_sample)
    print('D_real', D_real,'\n', 'D_fake', D_fake)
    print('D_logit_real', D_logit_real, '\n', 'D_logit_fake', D_logit_fake)
    break
    